<a href="https://colab.research.google.com/github/infinity-99/My-projects/blob/master/Rock_Paper_Scissors_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
pip install -q tensorflow tensorflow.datasets

Import Necessary Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

Find Datasets

In [ ]:
tfds.list_builders()

Get Information on the Data

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
info

Prepare rock, paper, scissors Data

In [ ]:
ds_train = tfds.load(name='rock_paper_scissors', split='train')
ds_test = tfds.load(name='rock_paper_scissors', split = 'test')

## Show some Examples

In [ ]:
fig = tfds.show_examples(ds_train, info)

## Additional Data Prep

In [ ]:
train_images = np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example['label'].numpy() for example in ds_train])

test_images = np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example['label'].numpy() for example in ds_test])

In [ ]:
train_images = train_images.reshape(2520, 300, 300, 1)
test_images = test_images.reshape(372, 300, 300, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

In [ ]:
train_images[0].shape

(300, 300, 1)

## Train a network (basic approach)

In [ ]:
model = keras.Sequential([
  keras.layers.Flatten(),
  keras.layers.Dense(512, input_shape=(300, 300, 1), activation='relu'),
  keras.layers.Dense(256, activation = 'relu'),
  keras.layers.Dense(3, activation = 'softmax')
])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)  

In [ ]:
model.evaluate(test_images, test_labels)

## Train a Network (convolutional approach)

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation= 'relu', input_shape = (300, 300, 1)),
  keras.layers.Conv2D(32, 3, activation = 'relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation = 'softmax')
])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)

In [ ]:
model.evaluate(test_images, test_labels)

## A Better Convolutional Network

In [ ]:
model = keras.Sequential([
  keras.layers.AveragePooling2D(6,3, input_shape = (300, 300, 1)),
  keras.layers.Conv2D(64, 3, activation= 'relu'),
  keras.layers.Conv2D(32, 3, activation = 'relu'),
  keras.layers.MaxPool2D(2, 2),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(3, activation = 'softmax')
])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)

In [ ]:
model.evaluate(test_images, test_labels)

## Hyperparameter Tuning

In [ ]:
pip install -U keras.tuner

In [ ]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6, 3, input_shape = (300, 300, 1)))

  for i in range(hp.Int("Conv Layers", min_value=0, max_value=3)):

  model.add(keras.layers.Conv2D(64, 3, activation='relu'))
  model.add(keras.layers.Conv2D(32, 3, activation='relu'))

  model.add(keras.layers.MaxPool2D(2, 2))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(hp.Choice('Dense layer', [64, 128, 256, 512, 1024]), activation='relu'))

  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

  return model

tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials=32,
)

 
tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs = 10, batch_size = 32)

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(test_images, test_labels)

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()